# Chapter 2. An Array of Sequences
---

## ToC

[Objectives](#objectives)

1. [Overview](#overview)
2. [List Comprehension and Generator Expressions](#list-comprehension-and-generator-expressions)  
    2.1. [Listcomps VS map and filter](#listcomps-vs-map-and-filter)  
    2.2. [Cartesian Products](#cartesian-products)  
    2.3. [Generator Expressions](#generator-expressions)
3. [Tuples are not just immutable lists](#tuples-are-not-just-immutable-lists)  
    3.1. [Tuples as Records](#tuples-as-records)  
    3.2. [Tuples as Immutable Lists](#tuples-as-immutable-lists)  
    3.3. [Comparing Tuple and List Methods](#comparing-tuple-and-list-methods)
4. [Unpacking Sequences and Iterables](#unpacking-sequences-and-iterables)  
    4.1. [Using * to Grab Excess Items](#using-*-to-grab-excess-items)  
    4.2. [Unpacking with * in Function Calls and Sequence Literals](#unpacking-with-*-in-function-calls-and-sequence-literals)  
    4.3. [Nested Unpacking](#nested-unpacking)  

## Objectives
- List comprehensions and the basics of generator expressions
- Using tuples as records versus using tuples as immutable lists
- Sequence unpacking and sequence patterns
- Reading from slices and writing to slices
- Specialized sequence types, like arrays and queues

## Overview

Python inherited from ABC the uniform handling of sequences. Strings, lists, byte
sequences, arrays, XML elements, and database results share a rich set of common
operations, including iteration, slicing, sorting, and concatenation.

**Sequence Types (item type)**

- *Container sequences*: Can hold items of different types, including nested containers. Some examples:
`list`, `tuple`, and `collections.deque`.

- *Flat sequences*: Hold items of one simple type. Some examples: `str`, `bytes`, and `array.array`.

**Sequence Types (mutability)**
- *Mutable sequences*: e.g., list, bytearray, array.array, and collections.deque.
- *Immutable sequences*: e.g. tuple, str, and bytes.

![Figure 11](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/11.PNG)

Thus, flat sequences are more compact, but they are limited to holding primitive
machine values like bytes, integers, and floats.

![Figure 12](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/12.PNG)

![Figure 13](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/13.PNG)

In [2]:
from collections import abc
print(issubclass(tuple, abc.Sequence))
print(issubclass(list, abc.MutableSequence))
print(issubclass(tuple, abc.MutableSequence))

True
True
False


## List Comprehensions and Generator Expressions

- list comprehension -> for lists
- generator expression -> for other kinds of sequences

![Figure 14](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/14.PNG)

for Python, as for English, there are no hard-and-fast rules for clear
writing. Use your best judgement on when to use listcomps; as a rule of thumb, if it spans more than two
lines, it is probably best to break it apart or rewrite it as a plain old `for` loop.

![Figure 15](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/15.PNG)

In [21]:
# multiline example
my_dict = {
    "name": "Alice",
    "age": 30,
    "location": "Wonderland",
}

### Listcomps Versus map and filter

Listcomps do everything the `map` and `filter` functions do, without the contortions of
the functionally challenged Python `lambda`.

In [6]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [7]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

In [9]:
symbols = '$¢£¥€¤'
list(filter(lambda c: c > 127, map(ord, symbols)))

[162, 163, 165, 8364, 164]

Comparing speed of each: [Github Repo](https://fpy.li/2-1)

### Cartesian Products

**3 Methods**

In [10]:
# Method I
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [12]:
# Method II
for color in colors:
    for size in sizes:
        print((color, size))

('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')


In [16]:
# Method III
tshirts = [(color, size) for size in sizes for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

### Generator Expressions

To initialize tuples, arrays, and other types of sequences, you could also start from a listcomp, but a genexp saves memory because it yields items one by one using the iterator protocol instead of building a whole list just to feed another constructor.

In [17]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [18]:
import array
array.array('I', (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

In [19]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
for tshirt in (f'{c} {s}' for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


Here the six-item list of T-shirts is never built in memory: the genexp feeds the for loop producing
one item at a time. If the two lists used in the Cartesian product had a thousand items each, using a genexp would save the cost of building a list with a million items just to feed the for loop

![Figure 16](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/16.PNG)

#### Memory Comparison

In [20]:
import sys

colors = ['black', 'white'] * 1000   # 2000 items
sizes = ['S', 'M', 'L'] * 1000       # 3000 items

# Cartesian product with list comprehension
tshirts_complist = [(c, s) for c in colors for s in sizes]
print("List size in memory:", sys.getsizeof(tshirts_complist))

# Cartesian product with generator expression
tshirts_genexp = ((c, s) for c in colors for s in sizes)
print("Generator size in memory:", sys.getsizeof(tshirts_genexp))

List size in memory: 49441368
Generator size in memory: 216


## Tuples Are Not Just Immutable Lists

Tuples do double duty: they can be used as immutable lists and also as records with no field names.
We often think of records as data structures with named fields.

### Tuples as Records

Tuples hold records: each item in the tuple holds the data for one field, and the position of the item gives its meaning.
In this context, contrary to lists, the number of items (quantity) is usually fixed and their order is always important.

In following example ,in every expression, sorting the tuple would destroy the information because the meaning of each field is given by its
position in the tuple.

In [22]:
lax_coordinates = (33.9425, -118.408056)
city, year, pop, chg, area = ('Tokyo', 2003, 32_450, 0.66, 8014)

traveler_ids = [
                ('USA', '31195855'), 
                ('BRA', 'CE342567'), 
                ('ESP', 'XDA205856')
            ]
for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


In [23]:
for country, _ in traveler_ids:
    print(country)

USA
BRA
ESP


![Figure 17](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/17.PNG)

### Tuples as Immutable Lists

**Advatnages**
- Clarity: When you see a tuple in code, you know its length will never change.
- Performance: A tuple uses less memory than a list of the same length, and it allows Python
to do some optimizations.

![Figure 18](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/18.PNG)

In [24]:
a = (10, 'alpha', [1, 2])
b = (10, 'alpha', [1, 2])
a == b

True

In [25]:
b[-1].append(99)
a == b

False

In [26]:
b

(10, 'alpha', [1, 2, 99])

Tuples with mutable items can be a source of bugs.
an object is only hashable if its value cannot ever change.
An unhashable tuple cannot be inserted as a `dict` key, or a `set` element.

#### Test Hashability

In [29]:
def fixed(o):
    try:
        hash(o)
    except TypeError:
        return False
    return True

tf = (10, 'alpha', (1, 2))
tm = (10, 'alpha', [1, 2])
print(fixed(tf))
print(fixed(tm))

True
False


Are tuples more efficient than lists in Python?
Answer: [StackOverflow](https://fpy.li/2-3)

### Comparing Tuple and List Methods

When using a tuple as an immutable variation of list, it is good to know how similar their APIs are. `tuple` supports all list methods that do not involve adding or removing items, with one exception—tuple lacks the
`__reversed__` method.

![Figure 19](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/19.PNG)

## Unpacking Sequences and Iterables

Unpacking is important because it avoids unnecessary and error-prone use of
indexes to extract elements from sequences. Also, unpacking works with any iterable
object as the data source—including iterators, which don’t support index notation
([]).

### Applications

**Parallel Assignment**

In [2]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates # unpacking
print(latitude)
print(longitude)

33.9425
-118.408056


**Swapping the Values of Variables**

In [3]:
a = 2
b = 3
b, a = a, b
print(a)
print(b)

3
2


**Refixing an Argument with * when Calling a Function**

In [ ]:
divmod(20, 8)

(2, 4)

In [ ]:
t = (20, 8)
divmod(*t)

(2, 4)

In [7]:
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

**Return Multiple Values**

In [13]:
import os
_, filename = os.path.split('/home/Hamed/.ssh/id_rsa.pub')
filename

'id_rsa.pub'

### Using * to Grab Excess Items

Defining function parameters with `*args` to grab arbitrary excess arguments is a classic Python feature.

In [14]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [16]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In [17]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

In [19]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

### Unpacking with * in Function Calls and Sequence Literals

In [20]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest

fun(*[1, 2], 3, *range(4, 7))

(1, 2, 3, 4, (5, 6))

**Defining `list`, `tuple`, or `set` Literals**

In [26]:
range(4)

range(0, 4)

In [ ]:
*range(4), 4

(0, 1, 2, 3, 4)

In [22]:
[*range(4), 4]

[0, 1, 2, 3, 4]

In [23]:
{*range(4), 4, *(5, 6, 7)}

{0, 1, 2, 3, 4, 5, 6, 7}

### Nested Unpacking

In [27]:
metro_areas = [
('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for name, _, _, (lat, lon) in metro_areas:
        if lon <= 0:
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

if __name__ == '__main__':
    main()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


The target of an unpacking assignment can also be a list, but good use cases are rare. If you have a database query that returns a single record (e.g., the SQL code has a LIMIT 1 clause), then you can unpack and at the same time make sure there’s only one result with this code

In [ ]:
# [record] = query_returning_single_row()
# If the record has only one field, you can get it directly, like this:
# [[field]] = query_returning_single_row_with_single_field() 

## Pattern Matching with Sequences

![Figure 20](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/20.PNG)

[Structural Pattern Matching Documentation](https://fpy.li/2-6)